# Демонстрация использования класса `OscillogramToCsvConverter`

Этот ноутбук показывает, как использовать класс `OscillogramToCsvConverter` из `raw_to_csv.raw_to_csv` для преобразования файлов COMTRADE в формат CSV. Этот класс является основным компонентом для подготовки данных для последующего использования, например, в машинном обучении.

Основные возможности, которые будут продемонстрированы:
- Базовое преобразование в CSV.
- Преобразование с вырезанием интересующих областей (вокруг событий, отмеченных ML-сигналами).
- Создание "упрощенного" CSV с агрегированными метками событий.

**Важно**: Этот ноутбук использует образцы файлов из директории `tests/sample_data/`. Для демонстрации файлы копируются во временную рабочую директорию (`temp_converter_notebook_data/...`). Убедитесь, что тестовые данные были сгенерированы запуском скрипта `tests/test_data_setup.py`.

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import json 
import csv # For creating sample norm.csv if needed, though we'll copy
import sys

# --- Пути к данным для тестирования ---
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path:
    sys.path.append(module_path)
    
from core.oscillogram import Oscillogram
from normalization.normalization import OscillogramNormalizer
from raw_to_csv.raw_to_csv import OscillogramToCsvConverter

base_sample_data_dir = os.path.join(module_path, "tests", "sample_data")
comtrade_samples_path = os.path.join(base_sample_data_dir, "comtrade_files")
config_samples_path = os.path.join(base_sample_data_dir, "config_files")

# --- Создание временной рабочей директории для этого ноутбука ---
notebook_base_temp_dir_conv = "temp_converter_notebook_data"
notebook_source_dir_conv = os.path.join(notebook_base_temp_dir_conv, "source_oscillograms_conv")
notebook_output_dir_conv_csv = os.path.join(notebook_base_temp_dir_conv, "output_csvs")
notebook_config_files_dir_conv = os.path.join(notebook_base_temp_dir_conv, "config_files_conv")

if os.path.exists(notebook_base_temp_dir_conv):
    shutil.rmtree(notebook_base_temp_dir_conv)
os.makedirs(notebook_source_dir_conv)
os.makedirs(notebook_output_dir_conv_csv)
os.makedirs(notebook_config_files_dir_conv)

# --- Копирование необходимых файлов для демонстрации ---
all_files_copied_conv = True
# 1. Осциллограммы
# sample_A from valid_cfg_dat_1
shutil.copy(os.path.join(comtrade_samples_path, "valid_cfg_dat_1_subdir", "sample_A.cfg"), notebook_source_dir_conv)
shutil.copy(os.path.join(comtrade_samples_path, "valid_cfg_dat_1_subdir", "sample_A.dat"), notebook_source_dir_conv)
    
# sample_B (for event cutting) from for_converter/
converter_sample_B_path = os.path.join(comtrade_samples_path, "for_converter")
sample_B_cfg_src = os.path.join(converter_sample_B_path, "sample_B.cfg")
sample_B_dat_src = os.path.join(converter_sample_B_path, "sample_B.dat")
if os.path.exists(sample_B_cfg_src) and os.path.exists(sample_B_dat_src):
    shutil.copy(sample_B_cfg_src, notebook_source_dir_conv)
    shutil.copy(sample_B_dat_src, notebook_source_dir_conv)
else:
    print(f"ОШИБКА: sample_B.cfg/dat (for event testing) не найдены в {converter_sample_B_path}")
    all_files_copied_conv = False
    # Create a dummy sample_B if missing
    print("Создание фиктивного sample_B.cfg/.dat для продолжения работы ноутбука.")
    dummy_cfg_b_content = ["StationB,DevB,1999","2,1A,1D","1,DummySigB,A,,V,1,0,0,0,0,1,1,P","2,EVENT_FOR_CUTOUT,,,0","50.0","1","1000,10","01/01/2023,00:00:00.000","01/01/2023,00:00:00.010","ASCII","1.0"]
    with open(os.path.join(notebook_source_dir_conv, "sample_B.cfg"), "w", encoding='utf-8') as f: 
        for line in dummy_cfg_b_content: f.write(line + "\n")
    with open(os.path.join(notebook_source_dir_conv, "sample_B.dat"), "w", encoding='utf-8') as f:
        f.write("1,0,0,0\n2,1000,0,1\n3,2000,0,0\n") # Minimal DAT

# 2. Файл коэффициентов нормализации
src_norm_path = os.path.join(config_samples_path, "sample_norm_coeffs.csv")
dest_norm_path_conv = os.path.join(notebook_config_files_dir_conv, "current_norm_coeffs.csv")
if os.path.exists(src_norm_path): shutil.copy(src_norm_path, dest_norm_path_conv)
else: print(f"ОШИБКА: sample_norm_coeffs.csv не найден в {config_samples_path}."); all_files_copied_conv = False
    
# 3. JSON словари для имен каналов
src_analog_json = os.path.join(config_samples_path, "dict_analog_names_sample.json")
dest_analog_json_conv = os.path.join(notebook_config_files_dir_conv, "current_analog_names.json")
if os.path.exists(src_analog_json): shutil.copy(src_analog_json, dest_analog_json_conv)
else: print(f"ОШИБКА: dict_analog_names_sample.json не найден в {config_samples_path}."); all_files_copied_conv = False

src_discrete_json = os.path.join(config_samples_path, "dict_discrete_names_sample.json")
dest_discrete_json_conv = os.path.join(notebook_config_files_dir_conv, "current_discrete_names.json")
if os.path.exists(src_discrete_json): 
    shutil.copy(src_discrete_json, dest_discrete_json_conv)
else: print(f"ОШИБКА: dict_discrete_names_sample.json не найден в {config_samples_path}."); all_files_copied_conv = False

if not all_files_copied_conv:
    print("Не все критичные файлы были скопированы. Демонстрация может быть неполной.")
else:
    print(f"Демонстрационные файлы и конфигурации скопированы во временные директории в: {notebook_base_temp_dir_conv}")
    print("Содержимое source_oscillograms_conv:")
    for item in os.listdir(notebook_source_dir_conv): print(f"- {item}")

## 1. Инициализация компонентов
Для работы `OscillogramToCsvConverter` требуется экземпляр `OscillogramNormalizer` (для нормализации данных) и пути к конфигурационным JSON-файлам, определяющим, как сырые имена каналов COMTRADE отображаются на стандартные имена в шинах (`dict_analog_names.json`, `dict_discrete_names.json`).

In [ ]:
# 1. Normalizer
normalizer_conv = OscillogramNormalizer(
    norm_coef_file_path=dest_norm_path_conv, 
    is_print_message=True # is_print_message is a param for OscillogramNormalizer constructor
)

# 2. OscillogramToCsvConverter
converter = OscillogramToCsvConverter(
    normalizer=normalizer_conv,
    raw_path=notebook_source_dir_conv,
    csv_path=notebook_output_dir_conv_csv,
    # uses_buses=['1'], # This parameter might not exist or be used differently, check class definition
    dict_analog_names_path=dest_analog_json_conv,
    dict_discrete_names_path=dest_discrete_json_conv,
    is_print_message=True, # Assuming this is still the param for OscillogramToCsvConverter
    show_progress_bars=True # Assuming this was added to OscillogramToCsvConverter
)
converter.number_periods = 2 
print("\nOscillogramToCsvConverter и зависимости инициализированы.")

## 2. Базовое преобразование в CSV (`create_csv`)

In [ ]:
output_csv_basic = "basic_converted_data.csv"
print(f"\nЗапуск базового преобразования в CSV (выход: {output_csv_basic})...")
df_basic = converter.create_csv(csv_name=output_csv_basic, is_cut_out_area=False, is_simple_csv=False)

if df_basic is not None and not df_basic.empty:
    print(f"\nПервые строки базового CSV ({output_csv_basic}):")
    try: from IPython.display import display; display(df_basic.head()) 
    except ImportError: print(df_basic.head())
    print(f"\nКолонки: {df_basic.columns.tolist()}")
else:
    print(f"Базовый CSV не был создан или пуст.")

## 3. Преобразование с вырезанием области события (`create_csv(is_cut_out_area=True)`)
Для этого в `osc_conv2_event.cfg` был добавлен сигнал `MLsignal_1_1_1_1`, который `OscillogramToCsvConverter` (через `rename_bus_columns` и `get_ml_signals`) должен распознать как событие "Оперативное включение, запуск двигателя" (`ML_1_1_1`). Метод `cut_out_area` оставит данные вокруг этого события.

In [ ]:
output_csv_cut = "cut_area_data.csv"
print(f"\nЗапуск преобразования с вырезанием области (выход: {output_csv_cut})...")
df_cut = converter.create_csv(csv_name=output_csv_cut, is_cut_out_area=True, is_simple_csv=False)

if df_cut is not None and not df_cut.empty:
    print(f"\nДанные из CSV с вырезанной областью ({output_csv_cut}):")
    print("Уникальные имена файлов в CSV (должны содержать '_event N'):")
    print(df_cut['file_name'].unique())
    print("\nПервые строки для примера:")
    try: from IPython.display import display; display(df_cut.head()) 
    except ImportError: print(df_cut.head())
    print(f"Всего строк: {len(df_cut)}")
else:
    print(f"CSV с вырезанной областью не был создан или пуст.")

## 4. Создание "упрощенного" CSV (`create_csv(is_simple_csv=True)`)
Упрощенный CSV агрегирует детальные ML-метки в более общие категории: `opr_swch`, `abnorm_evnt`, `emerg_evnt`, `normal`.

In [ ]:
output_csv_simple_main_name = "simple_data_main.csv"
print(f"\nЗапуск создания упрощенного CSV (основной выход: {output_csv_simple_main_name}, упрощенный: ..._simple.csv)...")
df_for_simple = converter.create_csv(csv_name=output_csv_simple_main_name, is_cut_out_area=False, is_simple_csv=True)

simple_csv_actual_path = os.path.join(notebook_output_dir_conv_csv, os.path.splitext(output_csv_simple_main_name)[0] + "_simple.csv")
if os.path.exists(simple_csv_actual_path):
    print(f"\nСодержимое УПРОЩЕННОГО CSV ({simple_csv_actual_path}):")
    df_simple_loaded = pd.read_csv(simple_csv_actual_path)
    try: from IPython.display import display; display(df_simple_loaded.head()) 
    except ImportError: print(df_simple_loaded.head())
    print(f"\nКолонки упрощенного CSV: {df_simple_loaded.columns.tolist()}")
    
    expected_simple_cols = ['opr_swch', 'abnorm_evnt', 'emerg_evnt', 'normal']
    original_ml_cols = converter.ml_all 
    
    has_simple_cols = all(col in df_simple_loaded.columns for col in expected_simple_cols)
    has_no_original_detailed_ml_cols = not any(col in df_simple_loaded.columns for col in original_ml_cols if col not in ['file_name'])

    print(f"Содержит новые агрегированные колонки: {has_simple_cols}")
    print(f"Удалены старые детальные ML-колонки: {has_no_original_detailed_ml_cols}")

else:
    print(f"Упрощенный CSV ({simple_csv_actual_path}) не был создан.")

## 5. Краткое упоминание `create_csv_for_PDR` и `create_csv_for_SPEF`

Класс `OscillogramToCsvConverter` также содержит методы `create_csv_for_PDR` и `create_csv_for_SPEF`. Они предназначены для создания специализированных CSV-файлов, адаптированных для анализа реле направления мощности (PDR) и однофазных замыканий на землю (SPEF) соответственно. Их использование аналогично `create_csv`, но может требовать специфической конфигурации сигналов в `dict_*.json` файлах или дополнительных входных данных (например, CSV-файла с результатами проверки сигналов для PDR).

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(notebook_base_temp_dir_conv):
        shutil.rmtree(notebook_base_temp_dir_conv)
        print(f"\nВременная директория {notebook_base_temp_dir_conv} удалена.")
    else:
        print(f"\nВременная директория {notebook_base_temp_dir_conv} не найдена для удаления.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {notebook_base_temp_dir_conv}: {e}")